### Imports

In [15]:
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime, timedelta
import random

### Data collection

#### Assume our data is stored as a CSV and we have time stamps

In [16]:
file = 'sensor_data.csv'
df = pd.read_csv(file)
df.describe()

,Temperature,Pressure,Humidity,BatteryCharge,NetCurrent,Tension,BatteryTemp,PM1.0,PM2.5,PM4.0,PM10.0,PM1.0_1/cm3,PM2.5_1/cm3,PM4.0_1/cm3,PM10.0_1/cm3,PMTypSize,CO2,StationID,MsgCounter,PacktCheckSum
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,24.962677,99882.328134,49.985628,90.070350,0.014147,3.855940,25.050224,25.427672,50.716258,79.028238,101.788141,496.718150,988.343967,1502.525450,1945.945363,4.832179,399.363332,5.068000,50.671000,129.125000
std,4.802120,5819.288828,17.385288,5.849052,0.511308,0.199038,5.113342,14.334648,29.722857,43.480751,57.210298,282.332887,571.531953,853.297579,1166.019610,2.809658,56.444571,2.562409,28.609111,73.814009
min,5.387999,90026.672790,20.079192,80.006195,-1.541281,3.500870,6.823999,0.126088,0.249545,0.007219,0.330498,1.091541,2.894441,0.330695,0.500555,0.035664,300.068874,1.000000,1.000000,0.000000
25%,21.741250,94868.658747,34.542572,85.037346,-0.336607,3.688469,21.753414,13.550406,24.395091,42.352885,51.432168,254.695790,507.920594,785.098926,914.646413,2.362809,353.334163,3.000000,26.000000,67.000000
50%,25.058604,99666.338090,49.779611,90.115871,0.032131,3.860045,25.092940,25.110206,52.743225,80.783542,101.621944,499.595442,986.584236,1497.046386,1996.390837,4.740923,401.552605,5.000000,51.000000,131.000000
75%,28.093162,105094.896478,65.729726,95.195068,0.361070,4.021928,28.418395,38.264855,76.189691,116.357282,151.727790,727.981179,1462.277557,2218.986747,2915.654854,7.284351,444.136268,7.000000,75.000000,193.000000
max,39.570546,109966.063115,79.988309,99.998497,1.714455,4.199260,40.702115,49.867227,99.990098,149.948446,199.366952,999.450523,1996.260232,2998.795147,3998.035183,9.996956,499.588083,9.000000,99.000000,254.000000


### Cleaning Data

#### For now I am going to assume the data is in perfect shape. Once we actually get real data I am going to modify this

### Exploratory Data Analysis

In [ ]:
# # Define features and data types
# features = {
#     "timestamp": pd.Timestamp,  # Qualitative (date and time)
#     # Gas sensors (V)
#     "GasSensor_0": np.float32,
#     "GasSensor_1": np.float32,
#     "GasSensor_2": np.float32,
#     "GasSensor_3": np.float32,
#     "GasSensor_4": np.float32,
#     "GasSensor_5": np.float32,
#     "GasSensor_6": np.float32,
#     "GasSensor_7": np.float32,
#     "GasSensor_8": np.float32,
#     "GasSensor_9": np.float32,
#     "GasSensor_10": np.float32,
#     "GasSensor_11": np.float32,
# 
#     "Temperature": np.float32,  # Quantitative (°C)
#     "Pressure": np.float32,    # Quantitative (Pa)
#     "Humidity": np.uint8,      # Quantitative (%RH)
#     "Battery_Charge": np.uint8,  # Quantitative (%)
#     "Net_Current": np.float32,  # Quantitative (A)
#     "Tension": np.float32,     # Quantitative (V)
#     "Battery_Temp": np.float32, # Quantitative (°C)
# 
#     # Particulate matter (µg/m³)
#     "PM1.0": np.float32,
#     "PM2.5": np.float32,
#     "PM4.0": np.float32,
#     "PM10.0": np.float32,
# 
#     # Particulate matter (1/cm³)
#     "PM1.0_cm3": np.float32,
#     "PM2.5_cm3": np.float32,
#     "PM4.0_cm3": np.float32,
#     "PM10.0_cm3": np.float32,
# 
#     "PM_Typ_Size": np.float32,  # Quantitative (µm)
#     "CO2": np.float32,           # Quantitative (ppm)
# }

# Generate random data
num_datapoints = 100 

# temp = pd.Timestamp(year=2017, month=1, day=1, hour=12, second=0)

# add more columns later
fake_df = pd.DataFrame(columns=['Temperature', 'Pressure', 'Humidity', 'Battery_Charge', 'Net_Current', 'Tension'])

start_time = datetime(2024, 2, 20, 10, 0, 0)  # Adjust start date and time

# the idea is to create random data right now. Tighten up the range so it's only dealing with a particular kind of 
# weather for now. Some combinations are thermodynamically impossible if you arbitrarily choose property values
for i in range(num_datapoints):
    print('test')
    # fake_df['Temperature'] = random.uniform(0, 100) # degrees celsius
    # fake_df['Pressure'] = random.uniform(99284.51, 103421) # potential pressure values in earth
    # fake_df['Humidity'] = 
    # fake_df['Battery_Charge'] =
    # fake_df['Net_Current'] =
    # fake_df['Tension'] = 

#### Visualizing data

### Visualizing data